# BerryBox Berry Segmentation

## Run inferencing using a trained YOLOv8 model

This notebook is meant to be run on a Windows PC using a CPU

0. Install depedencies

In [1]:
# Install dependencies
%pip install -r requirements.txt

# Open ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.186  Python-3.9.18 torch-2.2.1+cpu CPU (Intel Core(TM) i7-10810U 1.10GHz)
Setup complete  (12 CPUs, 31.6 GB RAM, 473.6/951.3 GB disk)


In [2]:
# Set project directory and change directory
import os
import shutil
proj_dir = os.getcwd()
proj_dir
%ls

 Volume in drive C is OS
 Volume Serial Number is 5E84-0237

 Directory of c:\Users\jeffrey.neyhart\OneDrive - USDA\Documents\CranberryLab\Breeding\2022\Imaging\BerryBoxImaging\mxo_gh_2021

03/19/2024  06:24 PM    <DIR>          .
03/19/2024  06:24 PM    <DIR>          ..
03/19/2024  05:49 PM    <DIR>          __pycache__
01/01/1980  12:00 AM            13,618 functions.py
03/19/2024  05:43 PM    <DIR>          images
03/19/2024  06:26 PM            37,768 inference.ipynb
03/19/2024  06:26 PM    <DIR>          output
03/17/2024  04:41 PM                79 requirements.txt
               3 File(s)         51,465 bytes
               5 Dir(s)  512,929,542,144 bytes free


1. Import a YOLO model and export it using OpenVINO

In [3]:
# Set the path to the model
# model_path = os.path.join(proj_dir, "models/berrybox_best_20240316.pt")
model_path = "C:/Users/jeffrey.neyhart/OneDrive - USDA/Documents/Repositories/berryboxai/models/berrybox_inst_seg_nano_best_20240318.pt"
# OpenVino model path
ov_model_path = "../models/" + os.path.basename(model_path).replace(".pt", "_openvino_model")

# Attempt to find the openvino version of the model;
# If it does not exist, export the model
if not os.path.exists(ov_model_path):

    # Copy the model locally
    model_path_local = os.path.join("models", os.path.basename(model_path))
    shutil.copyfile(model_path, model_path_local)

    # Load the model with YOLO
    from ultralytics import YOLO
    model = YOLO(model_path_local)

    # Export the model using openVINO
    # model.export(format = "openvino", imgsz = 2048, half = True)
    model.export(format = "openvino", imgsz = (1344, 2016), half = True)

2. Import dependecies, set parameters
Note: Make sure functions.py is in the current working directory

In [4]:
from functions import * # load all functions
from ultralytics import YOLO
import os
import torch
import gc
import shutil
from PIL import Image

gc.collect()   # collect garbage

device = '0' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')


"""
------------------------------------------------------------------------------------
Set Directories
------------------------------------------------------------------------------------
"""
model_dir = ov_model_path # path to the model
image_dir = 'images' # path to the image folder
save_dir = 'output' # path to save the results

# shutil.rmtree(save_dir, ignore_errors=True)

"""
------------------------------------------------------------------------------------
Set Model Parameters (you can change these parameters to suit your needs)
------------------------------------------------------------------------------------
"""
model_params = {
    'project': save_dir, # project name
    'name': "berrybox_" + os.path.basename(proj_dir), # run name
    'save': False, # save image results
    'show_labels': True,   # hide labels
    'show_conf': True, # hide confidences
    'save_crop': False, # save cropped prediction boxes
    'line_width': 3, # bounding box line width
    'conf': 0.70, # confidence threshold
    'iou': 0.75, # NMS IoU threshold
    # 'imgsz': 2048,
    'imgsz': (1344, 2016),
    # 'imgsz': (640, 960),
    'exist_ok': False, # if True, it overwrites current 'name' saving folder
    'half': True, # use FP16 half-precision inference True/False
    'cache': False, # use cache images for faster inference
    'retina_masks': False, #use high resolution seg mask
    'device': device, # cuda device, i.e. 0 or 0,1,2,3 or cpu
    'verbose': True
}

# Load the model
model = YOLO(model_dir, task = "segment")


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Using device: cpu


3. Run Inference

In [5]:
print('Running inference and extracting features...')

# Create an empty pandas data frame
DF = pd.DataFrame()

# List images in the image dir
image_files = [x for x in os.listdir(image_dir) if ".JPG" or ".PNG" in x.upper()]
# Target model size
newH, newW = model_params["imgsz"]

# Iterate over the image files
for i, img_name in enumerate(image_files):
    
    # Read in the image and resize
    image = Image.open(image_dir + "/" + img_name).resize((newW, newH))
    
    # Run through the model
    results = model.predict(source = image, **model_params)
    result = results[0]

    # Process the results
    # Try color correction; skip if it doesn't work
    try:
        result, patch_size = color_correction(result)
    except:
        continue
    # Was "info" found?
    if any(result.boxes.cls == get_ids(result, 'info')[0]):
        QR_info = read_QR_code(result)
    else:
        print("No 'info' detected by the model.\n")
        QR_info = img_name
    # Get features
    df1 = get_all_features_parallel(result, name= 'berry')
    df2 = get_all_features_parallel(result, name= 'rotten')
    df = pd.concat([pd.DataFrame({'name': (['berry'] * df1.shape[0]) + (['rotten'] * df2.shape[0])}), pd.concat([df1, df2], ignore_index = True)], axis = 1)    
    w,_ = df.shape
    img_name = [img_name]*w
    QR_info = [QR_info]*w
    patch_size = [np.mean(patch_size)]*w
    indeces = list(range(w))
    # If indeces is length 0; warn that no berries were found
    if len(indeces) == 0:
        print('\033[1;33mNo berries were found in the image!\033[0m')
        continue

    df_fore = pd.DataFrame({'Image_name': img_name,
                            'ID': indeces,
                            'QR_info': QR_info,
                            'Patch_size': patch_size})

    df = pd.concat([df_fore, df], axis=1)
    DF = pd.concat([DF, df], axis=0, ignore_index=True)

    img_save_folder = os.path.join(save_dir, 'predictions')
    if not os.path.exists(img_save_folder):
        os.makedirs(img_save_folder)

    save_ROI_parallel(result, get_ids(result, 'berry'), os.path.join(img_save_folder, img_name[0]))

    print(f"\nImage {i+1} of {len(image_files)} processed." )
    
    
DF.to_csv(os.path.join(save_dir, 'features.csv'), index=False)
print('Done.')

gc.collect()    

Running inference and extracting features...


Loading ..\models\berrybox_inst_seg_nano_best_20240318_openvino_model for OpenVINO inference...

0: 1344x2016 1 ColorCard, 1 berry, 1 info, 808.6ms
Speed: 30.1ms preprocess, 808.6ms inference, 77.1ms postprocess per image at shape (1, 3, 1344, 2016)



Color correcting the image...

Initial mean DeltaE:  6.51957356834
Number of components:  10

Color correction complete.


Reading QR code...


QR code reading complete.



Extracting features: 100%|██████████| 1/1 [00:00<00:00,  2.99it/s]
Extracting features: 0it [00:00, ?it/s]



Image 1 of 1203 processed.


...


0: 1344x2016 1 ColorCard, 7 berrys, 1 info, 1 rotten, 319.0ms
Speed: 32.0ms preprocess, 319.0ms inference, 225.2ms postprocess per image at shape (1, 3, 1344, 2016)



Color correcting the image...

Initial mean DeltaE:  6.27435344209
Number of components:  10

Color correction complete.


Reading QR code...


QR code reading complete.



Extracting features: 100%|██████████| 1/1 [00:00<00:00,  6.96it/s]



Image 1203 of 1203 processed.
Done.


9